# Генерируем датасет


## Нужные импорты и методы

In [ ]:
import pandas as pd
import json
import os
from tqdm.notebook import tqdm
import re
from sklearn.model_selection import train_test_split
import csv

In [ ]:
INCOMING_FILE_PATH =   'D:\wb_scraped\gpt_parsed_wb_10000.json'
INCOMING_FILE_PATH_2 = 'D:\wb_scraped\gpt_parsed_wb_10000_2.json'

In [ ]:
def check_tag(s):
    tag = ''
    if s[:3] == '<@#':
        tag = s[3:]
        if '>' in tag:
            tag = tag[:tag.find('>')]
        else:
            return 'Not a tag'
    else:
        return 'Does not start with a tag'

    value = ''
    if len(s.split('>')) > 1:
        value = s.split('>')[1]
        if value[-2:] == '<@':
            value = value[:-2]
        else:
            return 'Does not end properly'
    return (tag, value)

## Проверки методов

In [ ]:
print(check_tag('<@#Максимальная диагональ ноутбука>Ноутбук<@>'))
print(check_tag('<@#Максимальная диагональ ноутбука>Ноутбук<@'))
print(check_tag('<@#Максимальная диагональ ноутбука>Ноутбук@>'))
print(check_tag('<@#Максимальная диагональ Ноутбук<@>'))

('Максимальная диагональ ноутбука', 'Ноутбук')
('Максимальная диагональ ноутбука', 'Ноутбук')
Does not end properly
Does not end properly


In [ ]:
i = 2

print(ids[i])
print(texts[i])
print(titles[i])
print(descriptions[i])
print(atrs[i])
print(tags_t[i])
print(tags_d[i])
print(original_tag_t[i])
print(original_tag_d[i])
print(len(tags_t[i]), len(tags_d[i]), texts[i].count('@')//2, len(atrs[i]))

NameError: name 'ids' is not defined

In [ ]:
cnt_answ = 0
for i in range(10000):
    cnt_t = len(tags_t[i]) + len(tags_d[i])
    if cnt_t == len(atrs[i]):
        print(i, len(tags_t[i]), len(tags_d[i]), texts[i].count('@')//2, len(atrs[i]))
        cnt_answ+=1
print(cnt_answ)

NameError: name 'tags_t' is not defined

## Получение данных

In [ ]:
bert_df = pd.read_json(INCOMING_FILE_PATH)
bert_df_2 = pd.read_json(INCOMING_FILE_PATH_2)
bert_df = pd.concat([bert_df, bert_df_2])
bert_df

,_id,id,uri,properties,prompt,description_gpt
0,662d0ad5c5c94b46a12985f9,100044021,https://www.wildberries.ru/catalog/100044021/d...,"{'Модель': 'LS-77', 'Вид наушников': 'беспрово...",Сгенерируй описание и оглавление товара на мар...,Оглавление: Беспроводные наушники LS-77\nОписа...
1,662d0ad5c5c94b46a12985fa,100051530,https://www.wildberries.ru/catalog/100051530/d...,{'Максимальная диагональ ноутбука': '18 дюймов...,Сгенерируй описание и оглавление товара на мар...,Оглавление: Ноутбук 18 дюймов серебристого цве...
2,662d0ad5c5c94b46a12985fb,100079382,https://www.wildberries.ru/catalog/100079382/d...,"{'Модель': 'Star 2.0 V3', 'Тип накопителя': 'S...",Сгенерируй описание и оглавление товара на мар...,Оглавление: Компьютер Star 2.0 V3 с процессоро...
3,662d0ad5c5c94b46a12985fc,100083164,https://www.wildberries.ru/catalog/100083164/d...,"{'Количество предметов в упаковке': '1 шт', 'М...",Сгенерируй описание и оглавление товара на мар...,Оглавление: Органайзер для маскировки и защиты...
4,662d0ad5c5c94b46a12985fd,100083207,https://www.wildberries.ru/catalog/100083207/d...,"{'Количество предметов в упаковке': '1 шт', 'М...",Сгенерируй описание и оглавление товара на мар...,Оглавление: Органайзер для маскировки и защиты...
...,...,...,...,...,...,...
9995,6638a9c55d4fc292b1cdbd91,99883256,https://www.wildberries.ru/catalog/99883256/de...,{'Модель': 'автомобильное зарядное устройство;...,Сгенерируй описание и оглавление товара на мар...,Оглавление: Автомобильное зарядное устройство ...
9996,6638a9c55d4fc292b1cdbd92,99680634,https://www.wildberries.ru/catalog/99680634/de...,"{'Модель': 'Star 2.0 V2', 'Тип накопителя': 'S...",Сгенерируй описание и оглавление товара на мар...,Оглавление: Компьютер игровой Star 2.0 V2\nОпи...
9997,6638a9c55d4fc292b1cdbd93,99871511,https://www.wildberries.ru/catalog/99871511/de...,{'Модель': 'беспроводные наушники i9S TWS; гар...,Сгенерируй описание и оглавление товара на мар...,Оглавление: Беспроводные наушники i9S TWS\nОпи...
9998,6638a9c55d4fc292b1cdbd94,99944846,https://www.wildberries.ru/catalog/99944846/de...,{'Модель': 'Колонка для караоке с подсветкой F...,Сгенерируй описание и оглавление товара на мар...,Оглавление: Портативная колонка для караоке с ...


In [ ]:
ids, texts, titles, descriptions, atrs, original_tag_t, tags_t, original_tag_d, tags_d = [],[],[],[],[],[],[],[],[]

i = 3
def split_bert_tags(s, atr, id):
    title = bert_df.values[i][5].split('Оглавление:')[1]
    title = title.split('Описание:')[0]
    description = bert_df.values[i][5].split('Описание:')[1]

    title_res = []
    descr_res = []
    title_ori = []
    descr_ori = []

    for tag in re.finditer('<@#[^>]+>[^<]+<@>', title):
        res = check_tag(tag.group())
        title_res.append(res)
        title_ori.append(tag.group())
        #print(tag.group(), tag.span(), res)

    for tag in re.finditer('<@#[^>]+>[^<]+<@>', description):
        res = check_tag(tag.group())
        descr_res.append(res)
        descr_ori.append(tag.group())
        #print(tag.group(), tag.span(), res)

    ids.append(id)
    texts.append(s)
    titles.append(title)
    descriptions.append(description)
    atrs.append(atr)
    tags_t.append(title_res)
    tags_d.append(descr_res)
    original_tag_t.append(title_ori)
    original_tag_d.append(descr_ori)
    #print(title_ori, descr_ori)

    #print(title)
    #print(description)
    #print(atrs)

    return(s,title,description,atr,title_res,descr_res)

split_bert_tags(bert_df.values[i][5], bert_df.values[i][3], bert_df.values[i][1])

('Оглавление: Органайзер для маскировки и защиты проводов\nОписание: <@#Количество предметов в упаковке>1 шт<@> органайзер для маскировки и защиты проводов изготовлен из <@#Материал изделия>пластик; бамбук; abs-пластик<@>. Его <@#Ширина предмета>15.5 см<@> обеспечивает удобство использования. Произведено в <@#Страна производства>Китай<@>. Вес товара с упаковкой составляет <@#Вес товара с упаковкой (г)>600 г<@>, без упаковки - <@#Вес товара без упаковки (г)>500 г<@>. Комплектация включает в себя <@#Комплектация>короб органайзер для маскировки и защиты проводов - 1 шт<@>. Держатели проводов выполнены в виде <@#Тип держателей проводов>органайзер; лоток; защелка<@>.',
 ' Органайзер для маскировки и защиты проводов\n',
 ' <@#Количество предметов в упаковке>1 шт<@> органайзер для маскировки и защиты проводов изготовлен из <@#Материал изделия>пластик; бамбук; abs-пластик<@>. Его <@#Ширина предмета>15.5 см<@> обеспечивает удобство использования. Произведено в <@#Страна производства>Китай<@>. В

In [ ]:
ids, texts, titles, descriptions, atrs, original_tag_t, tags_t, original_tag_d, tags_d = [], [], [], [],[],[],[],[],[]

for i in tqdm(range(len(bert_df.values))):
    split_bert_tags(bert_df.values[i][5], bert_df.values[i][3], bert_df.values[i][1])

print(len(ids), len(texts), len(titles), len(descriptions), len(atrs), len(tags_t), len(tags_d), len(original_tag_t), len(original_tag_d))

  0%|          | 0/20003 [00:00<?, ?it/s]

20003 20003 20003 20003 20003 20003 20003 20003 20003


## Дискриминатор и фильтрация



In [ ]:
def check_generated_regular(i):
    a = []
    text = texts[i]
    cnt_t = len(tags_t[i]) + len(tags_d[i])
    if True:#cnt_t == texts[i].count('@')//2:
        if cnt_t >= 10 or cnt_t >= len(atrs[i])*0.7:
            for tag in tags_t[i]:
                #print(tag)
                if tag[0] in atrs[i] and tag[0] not in a:
                    a.append((tag[0], atrs[i][tag[0]]))
            for tag in tags_d[i]:
                #print(tag)
                if tag[0] in atrs[i] and tag[0] not in a:
                    a.append((tag[0], atrs[i][tag[0]]))
        if len(a) >= 10 or len(a) >= len(atrs[i])*0.7:
            for j in range(len(original_tag_t[i])):
                text = text.replace(original_tag_t[i][j], tags_t[i][j][1])
            for j in range(len(original_tag_d[i])):
                text = text.replace(original_tag_d[i][j], tags_d[i][j][1])
            if '@' not in text:
                return (True, text, a)
            #else:
            #    print(text)
    #print(a)
    return (False, texts[i], atrs[i])

In [ ]:
check_generated_regular(2)

(True,
 'Оглавление: Компьютер Star 2.0 V3 с процессором AMD Athlon 3000G и видеокартой GeForce GTX 1650\nОписание: Компьютер Star 2.0 V3 оборудован SSD накопителем объемом 480 ГБ, работает под управлением операционной системы Windows 10 Pro. Имеет HDMI; Ethernet; USB для подключения к различным устройствам, а также Wi-Fi для беспроводной связи. Количество портов 8 шт и 2 шт выходов на наушники обеспечивают удобство использования. Компьютер оснащен AMD Athlon 3000G (3.5 ГГц) и дискретной; GeForce GTX 1650 видеокартой с объемом памяти 4 Гб. Оперативная память DDR4 и ее объем 16 Гб обеспечивают быструю и эффективную работу. Компьютер подходит для работы и игр, имеет стильный черный дизайн. Гарантийный срок 3 Года (36 месяцев). Произведен в России. Вес с упаковкой 10 кг, без упаковки 8 кг. Комплектация включает 1 кабель питания; 1 - Системный блок. Размеры упаковки: длина - 44 см, высота - 41 см, ширина - 26 см.',
 [('Модель', 'Star 2.0 V3'),
  ('Тип накопителя', 'SSD'),
  ('Объем накопит

In [ ]:
cnt_answ = 0
for i in tqdm(range(20003)):
    if check_generated_regular(i)[0] == True:
        cnt_answ+=1
print(cnt_answ)

  0%|          | 0/20003 [00:00<?, ?it/s]

16719


In [ ]:
data_text = []
data_atrs = []
ids_filted = []
for i in range(20003):
    a = check_generated_regular(i)
    if a[0] == True:
        data_text.append(a[1])
        data_atrs.append(a[2])
        ids_filted.append(ids[i])
print(len(data_text), len(data_atrs))

16719 16719


In [ ]:
data_text[1000]

'Оглавление: SSD накопитель Kingston SU750 256GB 2.5" SATA 6Gb/s 3D NAND\nОписание: SSD накопитель Kingston 256 ГБ 2.5" SATA 6 Гбит/с с 3D NAND. Максимальная скорость чтения 550 Мб/с и записи 520 Мб/с. Гарантийный срок 36 мес.. Произведено в Китай. Размеры упаковки: длина - 10 см, ширина - 7 см, высота - 3 см.'

In [ ]:
data_atrs[1000]

[('Модель', 'SU750'),
 ('Объем накопителя', '256 ГБ; 256 gb'),
 ('Форм-фактор накопителя', '2.5"'),
 ('Интерфейс', 'SATA 6 Гбит/с'),
 ('Тип памяти накопителя', '3D NAND'),
 ('Максимальная скорость чтения', '550 Мб/с'),
 ('Максимальная скорость записи', '520 Мб/с'),
 ('Гарантийный срок', '36 мес.'),
 ('Страна производства', 'Китай'),
 ('Длина упаковки', '10 см'),
 ('Ширина упаковки', '7 см'),
 ('Высота упаковки', '3 см')]

In [ ]:
ids_filted[1000]

122972341

In [ ]:
d = dict()
for i in data_atrs:
  for j in i:
    if j[0] in d:
      d[j[0]] += 1
    else:
      d[j[0]] = 1

with open('D:/wb_scraped/all_attributes_encounters.csv', 'w', encoding='cp1251') as f:
    w = csv.writer(f)
    w.writerows(list(zip(d.keys(), d.values())))
    #w.writerow(d.values())

#### Заменяем габариты на нужные

In [ ]:
ids_filted[320]

In [ ]:
texts[401]

In [ ]:
data_text[320]

In [ ]:
ids[401]

In [ ]:
def standart_size(a):
  package_size_atrs = ['Длина упаковки','Ширина упаковки','Высота упаковки']
  item_size_atrs = ['Длина предмета','Ширина предмета','Высота предмета','Глубина предмета', 'Толщина предмета']
  standart_size = ['(длина)', '(ширина)', '(высота)', '(глубина)', '(толщина)']

  atr_names = [x[0] for x in a]

  package_size = ''
  item_size = ''
  for i in range(len(package_size_atrs)):
    if package_size_atrs[i] in atr_names:
      package_size += a[find_in_arr(a, package_size_atrs[i])][1] + " " + standart_size[i] + ", "
      a.remove(a[find_in_arr(a, package_size_atrs[i])])
  if package_size != '':
    package_size = package_size[:-2]
    a.append(('Габариты упаковки', package_size))

  for i in range(len(item_size_atrs)):
    if item_size_atrs[i] in atr_names:
      item_size += a[find_in_arr(a, item_size_atrs[i])][1] + " " + standart_size[i] + ", "
      a.remove(a[find_in_arr(a, item_size_atrs[i])])
  if item_size != '':
    item_size = item_size[:-2]
    a.append(('Габариты предмета', item_size))

  return a

def find_in_arr(a, x):
  for i in range(len(a)):
    if a[i][0] == x:
      return i

In [ ]:
for i in range(len(data_atrs)):
  standart_size(data_atrs[i])

In [ ]:
for i in range(len(data_atrs)):
  if len(set([x[0] for x in data_atrs[i]])) != len(data_atrs[i]):
    print(len(data_atrs[i]), len(set([x[0] for x in data_atrs[i]])))
    print(data_atrs[i])
    new_arr = []
    for j in set([x[0] for x in data_atrs[i]]):
      for l in range(len(data_atrs[i])):
        if data_atrs[i][l][0] == j:
          new_arr.append(data_atrs[i][l])
          break
    print(new_arr)
    data_atrs[i] = new_arr

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



## Генерация val и train

In [ ]:
wb_data = {
    'id' : ids_filted,
    'text':data_text,
    'atrs':data_atrs
}
wb_df = pd.DataFrame(wb_data)

wb_df

,id,text,atrs
0,100051530,Оглавление: Ноутбук 18 дюймов серебристого цве...,"[(Максимальная диагональ ноутбука, 18 дюймов),..."
1,100079382,Оглавление: Компьютер Star 2.0 V3 с процессоро...,"[(Модель, Star 2.0 V3), (Тип накопителя, SSD),..."
2,100083164,Оглавление: Органайзер для маскировки и защиты...,"[(Количество предметов в упаковке, 1 шт), (Мат..."
3,100083207,Оглавление: Органайзер для маскировки и защиты...,"[(Количество предметов в упаковке, 1 шт), (Мат..."
4,100083211,Оглавление: Органайзер для маскировки и защиты...,"[(Количество предметов в упаковке, 1 шт), (Мат..."
...,...,...,...
16714,99875272,Оглавление: Музыкальная колонка MINI - Портати...,"[(Модель, Музыкальная колонка MINI - Портативн..."
16715,99680634,Оглавление: Компьютер игровой Star 2.0 V2\nОпи...,"[(Модель, Star 2.0 V2), (Тип накопителя, SSD),..."
16716,99871511,Оглавление: Беспроводные наушники i9S TWS\nОпи...,"[(Вид наушников, беспроводные вкладыши), (Врем..."
16717,99944846,Оглавление: Портативная колонка для караоке с ...,"[(Модель, Колонка для караоке с подсветкой FM,..."


In [ ]:
train, test = train_test_split(wb_df, test_size=0.375, random_state=42, shuffle=True)
val, test = train_test_split(test, test_size=1.0/6, random_state=42, shuffle=True)

In [ ]:
#train = train[:5000]
train

,id,text,atrs
15259,59484091,Оглавление: Сетевой фильтр TESSAN TS-329\nОпис...,"[(Модель, Сетевой фильтр TESSAN TS-329), (Цвет..."
10505,213309748,Оглавление: Телевизор 28H540S 28 дюйм\nОписани...,"[(Модель, 28H540S 28 дюйм.), (Цвет, черный), (..."
10094,212213778,"Оглавление: Монитор LG LMN24F680-S 23.8""\nОпис...","[(Модель, LMN24F680-S), (Диагональ, 23.8""), (Р..."
323,111105393,Оглавление: Аудио адаптер 2.5 мм Jack папа - R...,"[(Комплектация, 1 шт), (Разъем 1, 2.5 мм Jack ..."
1344,13355638,Оглавление: Кабель Lightning для iPhone\nОписа...,"[(Длина шнура, 100 см), (Цвет, белый), (Габари..."
...,...,...,...
11284,216083081,"Оглавление: Телевизор 32-G11 с диагональю 32"" ...","[(Модель, 32; 32-G11), (Диагональ, 32""; 32 дюй..."
11964,218720351,"Оглавление: Ноутбук Intel Celeron N5095A 15.6""...","[(Операционная система, Windows 10), (Диагонал..."
5390,165922466,"Оглавление: Чехол для ноутбука ""Розовая фуксия...","[(Цвет, ярко-розовый; розовая фуксия; фуксия),..."
860,120163774,Оглавление: Блок питания 24V для светодиодной ...,"[(Модель, Блок питания 24V; Блок питания для с..."


In [ ]:
#test = test[:3000]
test

,id,text,atrs
5211,164775740,"Оглавление: Караоке система ""Беспроводной микр...","[(Типоразмер элемента питания, от аккумулятора..."
15930,78193842,Оглавление: Карта памяти microSDXC Class 10 с ...,"[(Модель, карта памяти; micro SDXC; microSDHC)..."
8971,209690997,Оглавление: Совместимый пурпурный лазерный кар...,"[(Вид картриджа для принтера, Совместимый), (Ц..."
12876,221819179,Оглавление: Телевизор 32A5KQ Frameless черный\...,"[(Модель, 32A5KQ Frameless черный), (Тип матри..."
8351,208659712,Оглавление: Ноутбук Apple MacBook Air 13 с про...,"[(Модель, Apple MacBook Air 13), (Операционная..."
...,...,...,...
3315,150414330,Оглавление: Сетевой фильтр с USB портами и 5 р...,"[(Модель, пилот сетевой фильтр; сетевой фильтр..."
586,115231410,Оглавление: Игровая компьютерная мышь Glory бе...,"[(Длина мыши, 120 мм), (Радиус действия (м), 1..."
7912,180472794,Оглавление: Телевизор 43 Smart TV S1\nОписание...,"[(Модель, 43 Smart TV S1), (Диагональ, 43""), (..."
8794,209167990,Оглавление: Ноутбук ROG STRIX SCAR 17 X3D G733...,"[(Модель, ROG STRIX SCAR 17 X3D G733PYV-LL067W..."


In [ ]:
val

,id,text,atrs
14852,48938516,Оглавление: Флеш-накопитель USB 128GB Exployd ...,"[(Модель, Флеш-накопитель USB 128GB Exployd 62..."
4257,158141075,Оглавление: Ноутбук AZ-1601 с операционной сис...,"[(Модель, AZ-1601), (Серия ноутбуков, AZ), (Оп..."
5183,164671041,Оглавление: Телевизор 43F690TS-T2\nОписание: Т...,"[(Модель, 43F690TS-T2), (Диагональ, 43""), (Фор..."
5688,168484826,Оглавление: Ночник с пультом управления USB\nО...,"[(Вид разъема, USB), (Питание, USB), (Комплект..."
7625,178971261,"Оглавление: Телевизор 40A4K 40"" FHD Smart TV с...","[(Модель, 40A4K), (Диагональ, 40""), (Формат эк..."
...,...,...,...
8166,181845620,"Оглавление: Смартфон iPhone XR в корпусе 14, б...","[(Модель, iPhone XR в корпусе 14), (Цвет, бело..."
14216,33527956,Оглавление: Кабель micro-usb X40 силиконовая о...,"[(Комплектация, Кабель micro-usb - 1шт), (Моде..."
11236,215841367,"Оглавление: Телевизор Samsung 65"" 4K UHD Smart...","[(Диагональ, 65""), (Тип матрицы, IPS), (Разреш..."
8570,208873258,"Оглавление: Вес с упаковкой (кг) 5.01 кг, Длин...","[(Габариты упаковки, 62 см (длина), 41 см (шир..."


In [ ]:
train_full, train_part = train_test_split(train, test_size=0.4, random_state=42, shuffle=True)
test_full, test_part = train_test_split(test, test_size=1/3, random_state=42, shuffle=True)
val_full, val_part = train_test_split(val, test_size=1/3, random_state=42, shuffle=True)

In [ ]:
test_part

,id,text,atrs
10542,213636296,"Оглавление: Женский кошелек ""Черный блеск""\nОп...","[(Цвет, черный), (Вес товара с упаковкой (г), ..."
15302,59942709,"Оглавление: Флеш-накопитель USB 2.0 ""Speedy""\n...","[(Интерфейс, USB 2.0), (Гарантийный срок, 1 го..."
14964,5091955,"Оглавление: Фотобумага ""А4; 220гр"" (25 шт) - К...","[(Модель, А4; 220гр), (Страна производства, Ки..."
8076,179223839,Оглавление: Термопринтер BT instant printing\n...,"[(Габариты упаковки, 16 см (длина), 11 см (шир..."
4064,155799624,Оглавление: Игровая механическая клавиатура с ...,"[(Модель, Cherry MX blue; аксессуары для ПК и ..."
...,...,...,...
5595,167846432,Оглавление: Термопринтер 420B\nОписание: Термо...,"[(Модель, 420B), (Страна производства, Китай),..."
12622,220823933,"Оглавление: Монитор N2753 27"" Full HD IPS 75Гц...","[(Модель, N2753), (Диагональ, 27""), (Формат эк..."
9049,209792586,"Оглавление: Монитор Redmi A27 27"" IPS 1920х108...","[(Модель, P27FBB-RA; Redmi A27), (Диагональ, 2..."
6072,170675435,Оглавление: Игровой компьютер Аллигатор 1650 B...,"[(Модель, Аллигатор 1650 Bluetooth), (Тип нако..."


In [ ]:
test_full

,id,text,atrs
6044,170482861,Оглавление: Удлинитель SFT-50\nОписание: SFT-5...,"[(Модель, SFT-50), (Количество гнезд/розеток, ..."
657,116696143,"Оглавление: Игровой коврик для мыши ""Прямоугол...","[(Цвет, белый), (Покрытие поверхности коврика,..."
5512,166927979,Оглавление: Кулер для системы охлаждения CoolM...,"[(Модель, CoolMoon), (Максимальное число оборо..."
6407,172331068,Оглавление: Ноутбук Acer Aspire 5 A515-57 I382...,"[(Модель, A515-57 I382SUN), (Серия ноутбуков, ..."
6023,170430444,Оглавление: Смартфон Hot 30 зеленого цвета с д...,"[(Модель, Hot 30), (Цвет, зеленый), (Диагональ..."
...,...,...,...
5514,166940550,Оглавление: Музыкальный центр SM-MS4205; Напол...,"[(Модель, Музыкальный центр SM-MS4205; Напольн..."
1934,140200577,"Оглавление: USB-концентратор ""usb-to-4usb-hub""...","[(Габариты упаковки, 7 см (длина), 2 см (ширин..."
3254,149740949,"Оглавление: Стабилизатор напряжения ""PowerGuar...","[(Габариты упаковки, 26 см (длина), 16 см (шир..."
11838,218377102,Оглавление: Ноутбук 3520-3820 с операционной с...,"[(Модель, 3520-3820), (Операционная система, L..."


In [ ]:
train_part_atrs = []
for i in train_part['atrs'].values:
    train_part_atrs.append(i[:5])
train_part['new_atrs'] = train_part_atrs

In [ ]:
test_part_atrs = []
for i in test_part['atrs'].values:
    test_part_atrs.append(i[:5])
test_part['new_atrs'] = test_part_atrs

In [ ]:
val_part_atrs = []
for i in val_part['atrs'].values:
    val_part_atrs.append(i[:5])
val_part['new_atrs'] = val_part_atrs

In [ ]:
#system = 'Ты алгоритм высокого уровня, для извлечения информации об атрибутах товара из его описания.'
full_pmt = 'Извлеки атрибуты товара в формате json из описания и оглавления продукта на маркетплейсе. В ответ выведи атрибуты товара и их значения.'
part_pmt_1 = 'Извлеки атрибуты товара в формате json из описания и оглавления продукта на маркетплейсе. В ответ выведи атрибуты товара и их значения. Выдели только следующие атрибуты: '
part_pmt_2 = 'Если данный атрибут не представлен в тексте, напиши вместо его значения "n/a".'

In [ ]:
train_part

,id,text,atrs,new_atrs
2093,141554751,"Оглавление: Беспроводная Bluetooth колонка ""Фи...","[(Акустический тип, 1.0), (Тип колонки, активн...","[(Акустический тип, 1.0), (Тип колонки, активн..."
6227,171383544,Оглавление: Корпус для компьютера Infinity WT ...,"[(Модель, Infinity WT), (Размер корпуса для ко...","[(Модель, Infinity WT), (Размер корпуса для ко..."
12624,220859622,Оглавление: Ноутбук RedmiBook Pro 14″\nОписани...,"[(Модель, JYU4460CN), (Версия операционной сис...","[(Модель, JYU4460CN), (Версия операционной сис..."
12063,219089735,"Оглавление: Монитор Redmi Display X27G 27"" 192...","[(Модель, Redmi Display X27G), (Диагональ, 27""...","[(Модель, Redmi Display X27G), (Диагональ, 27""..."
9586,211125507,Оглавление: Смартфон Redmi 13C 128 ГБ белый\n\...,"[(Модель, Redmi 13C), (Частота обновления, 90 ...","[(Модель, Redmi 13C), (Частота обновления, 90 ..."
...,...,...,...,...
4243,158080866,Оглавление: Baseus Cafule Series Metal Data Ca...,"[(Модель, Baseus Cafule Series Metal Data Cabl...","[(Модель, Baseus Cafule Series Metal Data Cabl..."
1788,139197524,"Оглавление: Беспроводной геймпад PG-9078, черн...","[(Модель, PG-9078), (Совместимость, android; i...","[(Модель, PG-9078), (Совместимость, android; i..."
739,119055475,"Оглавление: Компьютер ""Игровой Intel Core i5-1...","[(Тип накопителя, SSD), (Объем накопителя SSD,...","[(Тип накопителя, SSD), (Объем накопителя SSD,..."
301,110739241,"Оглавление: Аккумуляторная батарея ""Китай"" 100...","[(Страна производства, Китай), (Емкость аккуму...","[(Страна производства, Китай), (Емкость аккуму..."


In [ ]:
test_part

,id,text,atrs,new_atrs
10542,213636296,"Оглавление: Женский кошелек ""Черный блеск""\nОп...","[(Цвет, черный), (Вес товара с упаковкой (г), ...","[(Цвет, черный), (Вес товара с упаковкой (г), ..."
15302,59942709,"Оглавление: Флеш-накопитель USB 2.0 ""Speedy""\n...","[(Интерфейс, USB 2.0), (Гарантийный срок, 1 го...","[(Интерфейс, USB 2.0), (Гарантийный срок, 1 го..."
14964,5091955,"Оглавление: Фотобумага ""А4; 220гр"" (25 шт) - К...","[(Модель, А4; 220гр), (Страна производства, Ки...","[(Модель, А4; 220гр), (Страна производства, Ки..."
8076,179223839,Оглавление: Термопринтер BT instant printing\n...,"[(Габариты упаковки, 16 см (длина), 11 см (шир...","[(Габариты упаковки, 16 см (длина), 11 см (шир..."
4064,155799624,Оглавление: Игровая механическая клавиатура с ...,"[(Модель, Cherry MX blue; аксессуары для ПК и ...","[(Модель, Cherry MX blue; аксессуары для ПК и ..."
...,...,...,...,...
5595,167846432,Оглавление: Термопринтер 420B\nОписание: Термо...,"[(Модель, 420B), (Страна производства, Китай),...","[(Модель, 420B), (Страна производства, Китай),..."
12622,220823933,"Оглавление: Монитор N2753 27"" Full HD IPS 75Гц...","[(Модель, N2753), (Диагональ, 27""), (Формат эк...","[(Модель, N2753), (Диагональ, 27""), (Формат эк..."
9049,209792586,"Оглавление: Монитор Redmi A27 27"" IPS 1920х108...","[(Модель, P27FBB-RA; Redmi A27), (Диагональ, 2...","[(Модель, P27FBB-RA; Redmi A27), (Диагональ, 2..."
6072,170675435,Оглавление: Игровой компьютер Аллигатор 1650 B...,"[(Модель, Аллигатор 1650 Bluetooth), (Тип нако...","[(Модель, Аллигатор 1650 Bluetooth), (Тип нако..."


## Сохранения файлов

### Train part и full

In [ ]:
def list_to_json(A):
  a = A
  #if len(str(A)) > 2300:
  #  a = A[:-10]
  #  print('YOOOOO')
  ans = '{'
  for i in a:
    ans += ' \"' + i[0].replace('"', '\"') + '": \"' + i[1].replace('"', '\"') + '\",'
  ans = ans[:-1] + ' }'
  return ans

list_to_json(train_full['atrs'].values[1][:5])

'{ "Модель": "Q27G2E", "Диагональ": "27"", "Формат экрана": "16:9", "Разрешение экрана": "2560x1440", "Игровая модель": "да" }'

In [ ]:
id_test, sys, us, bot = [], [], [], [] ## Инструкция, Запрос, Ответ
#for i in range(len(train_full)):
for i in range(700):
    id_test.append(train_full['id'].values[i])
    sys.append(full_pmt)
    us.append(train_full['text'].values[i])
    bot.append(list_to_json((train_full['atrs'].values[i])))

#for i in range(len(train_part)):
for i in range(300):
    s = ''
    for j in train_part['new_atrs'].values[i]:
        s = s + j[0] + ", "
    s = s[:-2] + ".\n"
    id_test.append(train_part['id'].values[i])
    sys.append(part_pmt_1 + s + part_pmt_2)
    us.append(train_part['text'].values[i])
    bot.append(list_to_json((train_part['new_atrs'].values[i])))

In [ ]:
id_test[0]

171366206

In [ ]:
bot[0]

'{ "Модель": "32LEM-1088/TS2C", "Диагональ": "32"", "Формат экрана": "16:9", "Тип матрицы": "TFT", "Разрешение экрана": "1366x768", "Угол обзора": "178 градусов", "Яркость": "250 кд/кв.м", "Контрастность": "1400:1", "Частота смены кадров (Гц)": "60 Гц", "Тип звуковой системы": "Dolby Audio", "Мощность акустики": "10 Вт", "Количество встроенных динамиков": "2 шт.", "Размер VESA": "100x100", "Гарантийный срок": "1 год", "Страна производства": "Беларусь", "Вес без упаковки (кг)": "3 кг", "Габариты упаковки": "80 см (длина), 51 см (ширина), 14 см (высота)", "Габариты предмета": "73 см (ширина), 43 см (высота), 8 см (глубина)" }'

In [ ]:
bot[-1]

'{ "Модель": "Nitro KG272M3bmiipx (UM.HX2EE.303)", "Диагональ": "27"", "Формат экрана": "16:9", "Разрешение экрана": "1920х1080", "Время отклика": "0.5 мс" }'

In [ ]:
from json import loads, dumps
print(json.dumps({"request" : [{"role": "system", "text": "something"}, {"role": "user", "text": "something"}], "response": "Something"}, sort_keys=True, indent=4))

In [ ]:
req = []
for i in range(len(sys)):
  req.append([{"role": "system", "text": sys[i]}, {"role": "user", "text": us[i]}])

In [ ]:
for i in range(len(sys)):
  if (len(bot[i]) > 2000):
    print(i, len(bot[i]))

In [ ]:
len(str(train_full['atrs'].values[297]))

In [ ]:
a_data = {
    'request': req,
    'response':bot
}

a_json = pd.DataFrame(a_data)

In [ ]:
from json import loads, dumps
result = a_json.to_json(orient="records")
parsed = loads(result)
#dumps(parsed, indent=4)
with open('train_1000.json', 'w', encoding='UTF-8') as file:
        json.dump(parsed, file, indent=2, ensure_ascii=False)

In [ ]:
pip install jsonlines

In [ ]:
len(us[297]) + len(bot[297]) + len(sys[297])

In [ ]:
import jsonlines
with jsonlines.open('train_1000.jsonl', mode='w') as writer:
  for i in range(len(sys)):
    writer.write({"request" : [{"role": "system", "text": sys[i]}, {"role": "user", "text": us[i]}], "response": bot[i]})

In [ ]:
a_data = {
    'id':id_test,
    'system':sys,
    'user':us,
    'bot':bot
}

a_csv = pd.DataFrame(a_data)

In [ ]:
a_csv.to_csv('D:/wb_scraped/test_entries.csv', index=False, encoding='utf-8')

### Test part и full

# Пример запроса на YandexGPT


In [1]:
## Get IAM-token
import requests
import json
oauth = 'y0_AgAEA7qkJRUzAATuwQAAAAEEnYUhAACdnAq_qGxNboLJCoNnI5hmVl_HpQ'
req = {"yandexPassportOauthToken":oauth}
res = requests.post("https://llm.api.cloud.yandex.net/foundationModels/v1/completion", json=req)
print(res.json)

<bound method Response.json of <Response [400]>>


In [2]:
iam_token = 't1.9euelZqZlo-XyJuajZaUkYqVkI6Viu3rnpWanYuTiYyNxonKz56UkZnGj5nl9PdbLAxN-e8rah3z3fT3G1sJTfnvK2od883n9euelZrNlJqUmsrPipDGzsmNio6Xle_8xeuelZrNlJqUmsrPipDGzsmNio6XlQ.hGTLyMK5DlnXzZCZi9nZX0LbaQeJgnn-vTJD_fdeiYVq-CPWX3f5Zv-zQ19t1ShjpnEpxoJlv9ynve-CM5sPAw'

In [3]:
import requests
id_cat = 'b1gdfvvdf7m12f6bij3p'
# bt1g16pe9onk9k3uns6m
# b1gdfvvdf7m12f6bij3p
req = {
        "modelUri": "gpt://" + id_cat + "/yandexgpt/latest",
        "completionOptions": {
            "stream": False,
            "temperature": 0.1,
            "maxTokens": "500"
        },
        "messages": [
            {
            "role": "system",
            "text": "Извлеки атрибуты товара в формате json из описания и оглавления продукта на маркетплейсе. В ответ выведи атрибуты товара и их значения."
            },
            {
            "text": "Оглавление: Телевизор 32LEM-1088/TS2C\nОписание: Телевизор 32LEM-1088/TS2C имеет диагональ 32\" и формат экрана 16:9. Тип матрицы TFT, а разрешение экрана составляет 1366x768. Угол обзора 178 градусов, а яркость достигает 250 кд/кв.м. Контрастность 1400:1, а частота смены кадров 60 Гц. Телевизор поддерживает цифровые тюнеры DVB-T2, DVB-S2, DVB-C, а также разрешение HD 720p. Встроенная звуковая система Dolby Audio обеспечивает мощность 10 Вт с 2 шт.. Коммуникации и мультимедиа осуществляются через порты HDMI и USB. Телевизор можно крепить на стену с помощью VESA-крепления размером 100x100. Гарантийный срок составляет 1 год, а страна производства - Беларусь. Вес телевизора без упаковки 3 кг, а его размеры составляют 73 см x 8 см x 43 см. Комплектация включает в себя телевизор, пульт ДУ, настольную подставку и документацию. Упаковка имеет размеры 80 см x 14 см x 51 см.",
            "role": "user"
            }
        ]
}
headers = {"Authorization" : "Bearer " + iam_token,
                   "x-folder-id": id_cat, }
res = requests.post("https://llm.api.cloud.yandex.net/foundationModels/v1/completion",
    headers=headers, json=req)
print(res.json())
print(res.text)

{'result': {'alternatives': [{'message': {'role': 'assistant', 'text': '```json\n{\n  "Диагональ": "32",\n  "Формат экрана": "16:9",\n  "Тип матрицы": "TFT",\n  "Разрешение экрана": "1366x768",\n  "Угол обзора": 178,\n  "Яркость": 250,\n  "Контрастность": 1400:1,\n  "Частота смены кадров": 60,\n  "Цифровые тюнеры": ["DVB-T2", "DVB-S2", "DVB-C"],\n  "Поддержка разрешения HD": "720p",\n  "Звуковая система": "Dolby Audio",\n  "Мощность звуковой системы": 10,\n  "Количество динамиков": 2,\n  "Коммуникации и мультимедиа": ["HDMI", "USB"],\n  "VESA-крепление": "100x100",\n  "Гарантийный срок": 1,\n  "Страна производства": "Беларусь",\n  "Вес без упаковки": 3,\n  "Размеры без упаковки": "73 см x 8 см x 43 см",\n  "Комплектация": ["телевизор", "пульт ДУ", "настольная подставка", "документация"],\n  "Упаковка": "80 см x 14 см x 51 см"\n}\n```'}, 'status': 'ALTERNATIVE_STATUS_FINAL'}], 'usage': {'inputTextTokens': '301', 'completionTokens': '298', 'totalTokens': '599'}, 'modelVersion': '07.03.20

In [4]:
print(res.json()['result']['alternatives'][0]['message']['text'])

```json
{
  "Диагональ": "32",
  "Формат экрана": "16:9",
  "Тип матрицы": "TFT",
  "Разрешение экрана": "1366x768",
  "Угол обзора": 178,
  "Яркость": 250,
  "Контрастность": 1400:1,
  "Частота смены кадров": 60,
  "Цифровые тюнеры": ["DVB-T2", "DVB-S2", "DVB-C"],
  "Поддержка разрешения HD": "720p",
  "Звуковая система": "Dolby Audio",
  "Мощность звуковой системы": 10,
  "Количество динамиков": 2,
  "Коммуникации и мультимедиа": ["HDMI", "USB"],
  "VESA-крепление": "100x100",
  "Гарантийный срок": 1,
  "Страна производства": "Беларусь",
  "Вес без упаковки": 3,
  "Размеры без упаковки": "73 см x 8 см x 43 см",
  "Комплектация": ["телевизор", "пульт ДУ", "настольная подставка", "документация"],
  "Упаковка": "80 см x 14 см x 51 см"
}
```


In [ ]:
{
  "modelUri": "gpt://b1gdfvvdf7m12f6bij3p/yandexgpt",
  "messages": [
    {
      "text": "Извлеки атрибуты товара в формате json из описания и оглавления продукта на маркетплейсе. В ответ выведи атрибуты товара и их значения.",
      "role": "system"
    },
    {
      "text": "Оглавление: Телевизор 32LEM-1088/TS2C\nОписание: Телевизор 32LEM-1088/TS2C имеет диагональ 32\" и формат экрана 16:9. Тип матрицы TFT, а разрешение экрана составляет 1366x768. Угол обзора 178 градусов, а яркость достигает 250 кд/кв.м. Контрастность 1400:1, а частота смены кадров 60 Гц. Телевизор поддерживает цифровые тюнеры DVB-T2, DVB-S2, DVB-C, а также разрешение HD 720p. Встроенная звуковая система Dolby Audio обеспечивает мощность 10 Вт с 2 шт.. Коммуникации и мультимедиа осуществляются через порты HDMI и USB. Телевизор можно крепить на стену с помощью VESA-крепления размером 100x100. Гарантийный срок составляет 1 год, а страна производства - Беларусь. Вес телевизора без упаковки 3 кг, а его размеры составляют 73 см x 8 см x 43 см. Комплектация включает в себя телевизор, пульт ДУ, настольную подставку и документацию. Упаковка имеет размеры 80 см x 14 см x 51 см.",
      "role": "user"
    }
  ],
  "completionOptions": {
    "stream": true,
    "maxTokens": {
      "value": 500
    },
    "temperature": {
      "value": 0.1
    }
  }
}

# Тест для YandexGPT


In [ ]:
id_test, sys, us, bot = [], [], [], [] ## Инструкция, Запрос, Ответ
#for i in range(len(train_full)):
for i in range(700):
    id_test.append(test_full['id'].values[i])
    sys.append(full_pmt)
    us.append(test_full['text'].values[i])
    bot.append(list_to_json((test_full['atrs'].values[i])))

#for i in range(len(train_part)):
for i in range(300):
    s = ''
    for j in test_part['new_atrs'].values[i]:
        s = s + j[0] + ", "
    s = s[:-2] + ".\n"
    id_test.append(test_part['id'].values[i])
    sys.append(part_pmt_1 + s + part_pmt_2)
    us.append(test_part['text'].values[i])
    bot.append(list_to_json((test_part['new_atrs'].values[i])))

In [ ]:
#id_test, sys, us, bot
csv_answer = {
        'id': id_test,
        'sys': sys,
        'text': us,
        'atrs': bot
        }

csv_answer = pd.DataFrame(csv_answer)
csv_answer.to_csv('D:/wb_scraped/tests/wb_1000_test.csv', index=False, encoding='windows-1251')

In [ ]:
path = 'D:/wb_scraped/tests/yandexFineTunedGptAnswers1000/'
iam_token = 't1.9euelZqSjZPInY6clMudmMnHl5fKye3rnpWanYuTiYyNxonKz56UkZnGj5nl9PcGTGlN-e93fyC03fT3RnpmTfnvd38gtM3n9euelZrNio-UyMyTkpHNx4rJxs-Mnu_8xeuelZrNio-UyMyTkpHNx4rJxs-Mng.6ybOCpWSFf50pMBnxQUxbbFPoSjTbTMcP4Q8JDq6GJGnco3HRAO1TZBdwwIsiRXt0_q1apemDqdk_Ci9xdNyDw'

In [5]:
import requests
id_cat = 'b1gdfvvdf7m12f6bij3p'
id_model = 'bt16ivhgc3fojljfrbst'

def get_yandex_response(pmt, txt):
    req = {
        #"modelUri": "gpt://" + id_cat + "/yandexgpt/latest",
        "modelUri": "ds://" + id_model,
        "completionOptions": {
            "stream": False,
            "temperature": 0.1,
            "maxTokens": "500"
        },
        "messages": [
            {
            "role": "system",
            "text": pmt
            },
            {
            "text": txt,
            "role": "user"
            }
        ]
    }
    headers = {"Authorization" : "Bearer " + iam_token,
                       "x-folder-id": id_cat, }
    res = requests.post("https://llm.api.cloud.yandex.net/foundationModels/v1/completion",
        headers=headers, json=req)
    return res

In [ ]:
import requests
id_cat = 'b1gdfvvdf7m12f6bij3p'
id_model = 'bt16ivhgc3fojljfrbst'

def get_yandex_response_async(pmt, txt):
    req = {
        #"modelUri": "gpt://" + id_cat + "/yandexgpt/latest",
        "modelUri": "ds://" + id_model,
        "completionOptions": {
            "stream": False,
            "temperature": 0.1,
            "maxTokens": "500"
        },
        "messages": [
            {
            "role": "system",
            "text": pmt
            },
            {
            "text": txt,
            "role": "user"
            }
        ]
    }
    headers = {"Authorization" : "Bearer " + iam_token,
                       "x-folder-id": id_cat}
    res = requests.post("https://llm.api.cloud.yandex.net/foundationModels/v1/completionAsync",
        headers=headers, json=req)
    return res

def get_yandex_response_desync(id):
    headers = {"Authorization" : "Bearer " + iam_token}
    res = requests.get("https://llm.api.cloud.yandex.net/operations/" + id,
        headers=headers)
    return res

In [6]:
res = get_yandex_response("Извлеки атрибуты товара в формате json из описания и оглавления продукта на маркетплейсе. В ответ выведи атрибуты товара и их значения.", "Оглавление: Телевизор 32LEM-1088/TS2C\nОписание: Телевизор 32LEM-1088/TS2C имеет диагональ 32\" и формат экрана 16:9. Тип матрицы TFT, а разрешение экрана составляет 1366x768. Угол обзора 178 градусов, а яркость достигает 250 кд/кв.м. Контрастность 1400:1, а частота смены кадров 60 Гц. Телевизор поддерживает цифровые тюнеры DVB-T2, DVB-S2, DVB-C, а также разрешение HD 720p. Встроенная звуковая система Dolby Audio обеспечивает мощность 10 Вт с 2 шт.. Коммуникации и мультимедиа осуществляются через порты HDMI и USB. Телевизор можно крепить на стену с помощью VESA-крепления размером 100x100. Гарантийный срок составляет 1 год, а страна производства - Беларусь. Вес телевизора без упаковки 3 кг, а его размеры составляют 73 см x 8 см x 43 см. Комплектация включает в себя телевизор, пульт ДУ, настольную подставку и документацию. Упаковка имеет размеры 80 см x 14 см x 51 см.")

In [ ]:
res = get_yandex_response_async(sys[375], us[375])

print(res.ok)
print(res.json())
print(res.text)

In [ ]:
res = get_yandex_response(sys[0], us[0])

print(res.ok)
print(res.json())
print(res.text)

In [ ]:
res = get_yandex_response_desync('d7qefi459eombap68hir')

print(res.ok)
print(res.json())
print(res.text)

In [7]:
res.text

'{"result":{"alternatives":[{"message":{"role":"assistant","text":"{ \\"Модель\\": \\"32LEM-1088/TS2C\\", \\"Диагональ\\": \\"32\\"\\", \\"Формат экрана\\": \\"16:9\\", \\"Тип матрицы\\": \\"TFT\\", \\"Разрешение экрана\\": \\"1366x768\\", \\"Угол обзора\\": \\"178 градусов\\", \\"Яркость\\": \\"250 кд/кв.м\\", \\"Контрастность\\": \\"1400:1\\", \\"Частота смены кадров (Гц)\\": \\"60 Гц\\", \\"Тип цифрового ТВ\\": \\"DVB-T2; DVB-S2; DVB-C\\", \\"Разрешение HD\\": \\"720p\\", \\"Мощность акустики\\": \\"10 Вт\\", \\"Количество встроенных динамиков\\": \\"2 шт.\\", \\"Коммуникации и мультимедиа\\": \\"HDMI; USB\\", \\"VESA-крепление\\": \\"100x100\\", \\"Гарантийный срок\\": \\"1 год\\", \\"Страна производства\\": \\"Беларусь\\", \\"Вес без упаковки (кг)\\": \\"3 кг\\", \\"Габариты упаковки\\": \\"80 см (длина), 51 см (ширина), 14 см (высота)\\", \\"Габариты предмета\\": \\"73 см (ширина), 43 см (высота), 8 см (глубина)\\" }"},"status":"ALTERNATIVE_STATUS_FINAL"}],"usage":{"inputTextToke

In [ ]:
with open(str(path) + str(id_test[0]) + '.json', 'w', encoding='UTF-8') as file:
        json.dump(res.json(), file, indent=2, ensure_ascii=False)
        print(f'Данные сохранены в ' + str(path) + str(id_test[0]) + '.json' )

In [ ]:
id_test[375]

In [ ]:
id_test.index(135504671)

In [ ]:
path

In [ ]:
import time
import random

for i in tqdm(range(0, len(id_test))):
    res = get_yandex_response_async(sys[i], us[i])
    if not res.ok:
        print(i)
        print(res.json())
        break

    with open(str(path) + 'async/' + str(id_test[i]) + 'async.json', 'w', encoding='UTF-8') as file:
        json.dump(res.json(), file, indent=2, ensure_ascii=False)
        print(f'Данные сохранены в ' + str(path) + 'async/' + str(id_test[i]) + 'async.json' )

    sleep_for = random.uniform(1, 2)
    print('sleeping for', sleep_for)
    time.sleep(sleep_for)


In [ ]:
arr = os.listdir('D:/wb_scraped/tests/yandexGptAnswers1000/async/')
gpt_ids = []
gpt_files = []

for f in arr:
    #print(f,'!'*100)
    with open('D:/wb_scraped/tests/yandexGptAnswers1000/async/' + f, 'r', encoding='utf-8') as myfile:
            data=myfile.read()

    # parse file
    obj = json.loads(data)
    data = obj
    gpt_ids.append(obj['id'])
    gpt_files.append(f.split('async')[0])
    #print(f, f.split('async')[0])
    #print(obj['id'])

In [ ]:
path = 'D:/wb_scraped/tests/yandexGptAnswers1000/'

In [ ]:
import time
import random

for i in tqdm(range(len(gpt_files))):
    sleep_for = random.uniform(1, 2)
    print('sleeping for', sleep_for)
    time.sleep(sleep_for)

    res = get_yandex_response_desync(gpt_ids[i])
    if not res.ok:
        print(i)
        print(res.json())
        break

    with open(str(path) + 'desync/' + str(gpt_files[i]) + 'desync.json', 'w', encoding='UTF-8') as file:
        json.dump(res.json(), file, indent=2, ensure_ascii=False)
        print(f'Данные сохранены в ' + str(path) + 'async/' + str(id_test[i]) + 'async.json' )

In [ ]:
arr = os.listdir('D:/wb_scraped/tests/yandexFineTunedGptAnswers1000/async/')
gpt_ids = []
gpt_files = []

for f in arr:
    #print(f,'!'*100)
    with open('D:/wb_scraped/tests/yandexFineTunedGptAnswers1000/async/' + f, 'r', encoding='utf-8') as myfile:
            data=myfile.read()

    # parse file
    obj = json.loads(data)
    data = obj
    gpt_ids.append(obj['id'])
    gpt_files.append(f.split('async')[0])
    #print(f, f.split('async')[0])
    #print(obj['id'])

In [ ]:
len(gpt_files)

In [ ]:
path = 'D:/wb_scraped/tests/yandexFineTunedGptAnswers1000/'

In [ ]:
from huggingface_hub import HfApi
api = HfApi()

api.upload_folder(
    folder_path="D:/wb_scraped/gguf/saiga_gguf",
    repo_id="BaronFonMonc/Saiga_7b_wb_gguf_16",
    repo_type="space",
)

In [ ]:
import time
import random

for i in tqdm(range(len(gpt_files))):
    sleep_for = random.uniform(1, 2)
    print('sleeping for', sleep_for)
    time.sleep(sleep_for)

    res = get_yandex_response_desync(gpt_ids[i])
    if not res.ok:
        print(i)
        print(res.json())
        break

    with open(str(path) + 'desync/' + str(gpt_files[i]) + 'desync.json', 'w', encoding='UTF-8') as file:
        json.dump(res.json(), file, indent=2, ensure_ascii=False)
        print(f'Данные сохранены в ' + str(path) + 'desync/' + str(gpt_files[i]) + 'desync.json' )

НЕ ИСПОЛЬЗУЙ СИНХРОННЫЕ ЗАПРОСЫ

In [ ]:
#id_test, sys, us, bot
responses = []

In [ ]:
len(id_test)

100

In [ ]:
import requests
id_cat = 'b1gdfvvdf7m12f6bij3p'
id_model = 'bt16ivhgc3fojljfrbst'

def get_yandex_response(pmt, txt):
    req = {
        #"modelUri": "gpt://" + id_cat + "/yandexgpt/latest",
        "modelUri": "ds://" + id_model,
        "completionOptions": {
            "stream": False,
            "temperature": 0.1,
            "maxTokens": "500"
        },
        "messages": [
            {
            "role": "system",
            "text": pmt
            },
            {
            "text": txt,
            "role": "user"
            }
        ]
    }
    headers = {"Authorization" : "Bearer " + iam_token,
                       "x-folder-id": id_cat, }
    res = requests.post("https://llm.api.cloud.yandex.net/foundationModels/v1/completion",
        headers=headers, json=req)
    return res

In [ ]:
path = 'D:/wb_scraped/tests/yandexFineTunedGptAnswers1000/Real/'
iam_token = 't1.9euelZqelMaPk5KQyJmbnpCeksuUj-3rnpWanYuTiYyNxonKz56UkZnGj5nl8_cMQE1N-e89GStj_N3z90xuSk357z0ZK2P8zef1656VmpSJls6VkJCWj82JkMyYyYyX7_zF656VmpSJls6VkJCWj82JkMyYyYyX.Podl-zgCBHNdHPHrcLT05P7lfIL3KFZgv3Kkavq8DotTSwcf2dR9bH40n2mXtbrjr_DC8PW2p6rTdkYvdwM8DA'

In [ ]:
import time
import random

for i in tqdm(range(89, len(id_test))):
    #if size[i]>5:
    #    continue
    sleep_for = random.uniform(2, 10)
    print('sleeping for', sleep_for)
    time.sleep(sleep_for)

    res = get_yandex_response(sys[i], us[i])
    if not res.ok:
        print(i)
        print(res.json())
        break

    responses.append(res.json)

    with open(str(path) + str(id_test[i]) + '.json', 'w', encoding='UTF-8') as file:
        json.dump(res.json(), file, indent=2, ensure_ascii=False)
        print(f'Данные сохранены в ' + str(path) + str(id_test[i]) + '.json' )

  0%|          | 0/11 [00:00<?, ?it/s]

sleeping for 9.277837007245653
Данные сохранены в D:/wb_scraped/tests/yandexFineTunedGptAnswers1000/Real/209192056.json
sleeping for 5.686847225250473
Данные сохранены в D:/wb_scraped/tests/yandexFineTunedGptAnswers1000/Real/214887848.json
sleeping for 3.7389242170585844
Данные сохранены в D:/wb_scraped/tests/yandexFineTunedGptAnswers1000/Real/216539480.json
sleeping for 4.856633936446924
Данные сохранены в D:/wb_scraped/tests/yandexFineTunedGptAnswers1000/Real/159710827.json
sleeping for 9.8664506600118
Данные сохранены в D:/wb_scraped/tests/yandexFineTunedGptAnswers1000/Real/171086221.json
sleeping for 2.2771935464128097
Данные сохранены в D:/wb_scraped/tests/yandexFineTunedGptAnswers1000/Real/18917623.json
sleeping for 8.274826641383228
Данные сохранены в D:/wb_scraped/tests/yandexFineTunedGptAnswers1000/Real/222103531.json
sleeping for 4.953310038405123
Данные сохранены в D:/wb_scraped/tests/yandexFineTunedGptAnswers1000/Real/14411265.json
sleeping for 2.6306148012972104
Данные сох

In [ ]:
id_test, sys, us, bot = [], [], [], [] ## Инструкция, Запрос, Ответ
#for i in range(len(train_full)):
for i in range(300):
    #id_test.append(test_full['id'].values[i])
    sys.append(full_pmt)
    us.append(test_full['text'].values[i])
    bot.append(list_to_json((test_full['atrs'].values[i])))

#for i in range(len(train_part)):
for i in range(200):
    s = ''
    for j in test_part['new_atrs'].values[i]:
        s = s + j[0] + ", "
    s = s[:-2] + ".\n"
    #id_test.append(test_part['id'].values[i])
    sys.append(part_pmt_1 + s + part_pmt_2)
    us.append(test_part['text'].values[i])
    bot.append(list_to_json((test_part['new_atrs'].values[i])))

TypeError: can only concatenate str (not "tuple") to str

In [ ]:
from json import loads, dumps

a_data = {
    'system':sys,
    'user':us,
    'bot':bot
}

a_json = pd.DataFrame(a_data)

result = a_json.to_json(orient="records")
parsed = loads(result)
#dumps(parsed, indent=4)
with open('D:/wb_scraped/train.json', 'w', encoding='UTF-8') as file:
        json.dump(parsed, file, indent=2, ensure_ascii=False)

In [ ]:
id_test, sys, us, bot = [], [], [], [] ## Инструкция, Запрос, Ответ
#for i in range(len(train_full)):
for i in range(200):
    #id_test.append(test_full['id'].values[i])
    sys.append(full_pmt)
    us.append(train_full['text'].values[i])
    bot.append(list_to_json((train_full['atrs'].values[i])))

#for i in range(len(train_part)):
for i in range(100):
    s = ''
    for j in train_part['new_atrs'].values[i]:
        s = s + j[0] + ", "
    s = s[:-2] + ".\n"
    #id_test.append(test_part['id'].values[i])
    sys.append(part_pmt_1 + s + part_pmt_2)
    us.append(train_part['text'].values[i])
    bot.append(list_to_json((train_part['new_atrs'].values[i])))

In [ ]:
sys

['Извлеки атрибуты товара в формате json из описания и оглавления продукта на маркетплейсе. В ответ выведи атрибуты товара и их значения.',
 'Извлеки атрибуты товара в формате json из описания и оглавления продукта на маркетплейсе. В ответ выведи атрибуты товара и их значения.',
 'Извлеки атрибуты товара в формате json из описания и оглавления продукта на маркетплейсе. В ответ выведи атрибуты товара и их значения.',
 'Извлеки атрибуты товара в формате json из описания и оглавления продукта на маркетплейсе. В ответ выведи атрибуты товара и их значения.',
 'Извлеки атрибуты товара в формате json из описания и оглавления продукта на маркетплейсе. В ответ выведи атрибуты товара и их значения.',
 'Извлеки атрибуты товара в формате json из описания и оглавления продукта на маркетплейсе. В ответ выведи атрибуты товара и их значения.',
 'Извлеки атрибуты товара в формате json из описания и оглавления продукта на маркетплейсе. В ответ выведи атрибуты товара и их значения. Выдели только следующи

In [ ]:
bot

[{'Модель': 'Huawei B311-221',
  'Количество портов LAN': '1 шт.',
  'Количество портов WAN': '1 шт.',
  'Количество антенн': '1',
  'Длина предмета': '18 см',
  'Высота предмета': '2 см',
  'Ширина предмета': '18 см',
  'Тип антенны': 'внешняя',
  'Подключение 3G/4G модема': 'да',
  'Комплектация': 'роутер; антенна 5 м; сим-карта'},
 {'Модель': 'AZ-1507',
  'Диагональ экрана': '15.6"',
  'Тип матрицы': 'IPS',
  'Разрешение экрана': '1920x1080',
  'Процессор': 'Intel Celeron J4125 (2.0 ГГц)',
  'Количество ядер процессора': '4',
  'Тактовая частота процессора': '2.0ГГц',
  'Тип оперативной памяти': 'DDR4',
  'Объем оперативной памяти (Гб)': '8 Гб',
  'Объем накопителя SSD': '256 ГБ',
  'Беспроводные интерфейсы': 'Wi-Fi; Bluetooth'},
 {'Модель': 'AZ-1506',
  'Диагональ экрана': '15.6"',
  'Разрешение экрана': '1920x1080',
  'Процессор': 'Intel Celeron J4125',
  'Количество ядер процессора': '4',
  'Тактовая частота процессора': '2 ГГц',
  'Объем оперативной памяти (Гб)': '8 Гб',
  'Объе

In [ ]:
def list_to_json(A):
  a = A
  #if len(str(A)) > 2300:
  #  a = A[:-10]
  #  print('YOOOOO')
  ans = '{'
  for i in a:
    ans += ' \"' + i[0].replace('"', '\"') + '": \"' + i[1].replace('"', '\"') + '\",'
  ans = ans[:-1] + ' }'
  return ans


In [ ]:
def list_to_json_1(A):
  a = A
  #if len(str(A)) > 2300:
  #  a = A[:-10]
  #  print('YOOOOO')
  ans = '{'
  for i in a:
    ans += ' \"' + i.replace('"', '\"') + '": \"' + a[i].replace('"', '\"') + '\",'
  ans = ans[:-1] + ' }'
  return ans

In [ ]:
list_to_json_1(bot[0])

'{ "Модель": "Huawei B311-221", "Количество портов LAN": "1 шт.", "Количество портов WAN": "1 шт.", "Количество антенн": "1", "Длина предмета": "18 см", "Высота предмета": "2 см", "Ширина предмета": "18 см", "Тип антенны": "внешняя", "Подключение 3G/4G модема": "да", "Комплектация": "роутер; антенна 5 м; сим-карта" }'

In [ ]:
for i in range(len(bot)):
    bot[i] = list_to_json_1(bot[i])

In [ ]:
bot[0]

'{ "Модель": "Huawei B311-221", "Количество портов LAN": "1 шт.", "Количество портов WAN": "1 шт.", "Количество антенн": "1", "Длина предмета": "18 см", "Высота предмета": "2 см", "Ширина предмета": "18 см", "Тип антенны": "внешняя", "Подключение 3G/4G модема": "да", "Комплектация": "роутер; антенна 5 м; сим-карта" }'

In [ ]:
from json import loads, dumps

a_data = {
    'system':sys,
    'user':us,
    'bot':bot
}

a_json = pd.DataFrame(a_data)

result = a_json.to_json(orient="records")
parsed = loads(result)
#dumps(parsed, indent=4)
with open('D:/wb_scraped/val_real.json', 'w', encoding='UTF-8') as file:
        json.dump(parsed, file, indent=2, ensure_ascii=False)

---------------------------

In [ ]:
id_test, sys, us, bot = [], [], [], [] ## Инструкция, Запрос, Ответ

a = val_full#train_full
b = val_part#train_part
#for i in range(len(train_full)):
for i in range(len(a)):
    #id_test.append(test_full['id'].values[i])
    sys.append(full_pmt)
    us.append(a['text'].values[i])
    bot.append(list_to_json((a['atrs'].values[i])))

#for i in range(len(train_part)):
for i in range(len(b)):
    s = ''
    for j in b['new_atrs'].values[i]:
        s = s + j[0] + ", "
    s = s[:-2] + ".\n"
    #id_test.append(test_part['id'].values[i])
    sys.append(part_pmt_1 + s + part_pmt_2)
    us.append(b['text'].values[i])
    bot.append(list_to_json((b['new_atrs'].values[i])))

In [ ]:
print(len(a), len(b))

3483 1742


In [ ]:
print(len(id_test), len(sys), len(us), len(bot))

0 5225 5225 5225


In [ ]:
from json import loads, dumps

a_data = {
    'system':sys,
    'user':us,
    'bot':bot
}

a_json = pd.DataFrame(a_data)

result = a_json.to_json(orient="records")
parsed = loads(result)
#dumps(parsed, indent=4)
with open('D:/wb_scraped/val.json', 'w', encoding='UTF-8') as file:
        json.dump(parsed, file, indent=2, ensure_ascii=False)

# Тесты


In [ ]:
import pandas as pd
import json
import os
from tqdm.notebook import tqdm
import re
from sklearn.model_selection import train_test_split
import csv
import json
import os

arr = os.listdir('D:/wb_scraped/tests/RealData/Done')
ids = []
names = []
descriptions = []
formated_options = []
descr = []
size = []
clean = []

for i in range(2):
    for f in arr:
        #print(f,'!'*100)
        # read file
        with open('D:/wb_scraped/tests/RealData/Done/' + f, 'r', encoding='utf-8') as myfile:
            data=myfile.read()

        # parse file
        obj = json.loads(data)
        data = obj

        cnt = int(f.split('_')[0])
        ids.append(data[0])
        names.append(data[1])
        descriptions.append(data[2])
        formated_options.append(data[3])
        size.append(cnt)
        descr.append('Оглавление: ' + data[1] + '\nОписание: ' + data[2])

        s = descr[-1]
        s = s.replace('<@>', '')
        for tag in re.finditer('<@#[^>]+>', s):
            #print(tag.group())
            s = s.replace(tag.group(), '')
        clean.append(s)
        #for tag in re.finditer('<@>', s):
        #    print(tag)

    print(len(ids), len(names), len(descriptions), len(formated_options), len(size), len(descr))

d_data = {
        'ids' : ids,
        'description' : descr,
        'options' : formated_options,
        'size' : size,
        'clean' : clean
}

d_data = pd.DataFrame(d_data)

d_data

50 50 50 50 50 50
100 100 100 100 100 100


,ids,description,options,size,clean
0,143849214,Оглавление: Роутер <@#Модель>Huawei B311-221<@...,"{'Модель': 'Huawei B311-221', 'Количество порт...",10,Оглавление: Роутер Huawei B311-221 с дополните...
1,75963222,Оглавление: Ноутбук <@#Модель>AZ-1507<@> <@#Ди...,"{'Модель': 'AZ-1507', 'Диагональ экрана': '15....",11,"Оглавление: Ноутбук AZ-1507 15.6"" 4x2.0GHz, 8G..."
2,78369997,Оглавление: Ноутбук <@#Модель>AZ-1506<@> <@#Ди...,"{'Модель': 'AZ-1506', 'Диагональ экрана': '15....",11,"Оглавление: Ноутбук AZ-1506 15.6"" 4x2.0ГГц, 8G..."
3,205723877,Оглавление: Смартфон <@#Модель>OPPO F5 CPH1723...,"{'Модель': 'OPPO F5 CPH1723', 'Цвет': 'золотис...",12,Оглавление: Смартфон OPPO F5 CPH1723 6/128 GB ...
4,207323577,Оглавление: Ноутбук <@#Модель>Apple MacBook Ai...,"{'Модель': 'Apple MacBook Air 15 M2', 'Цвет': ...",14,Оглавление: Ноутбук Apple MacBook Air 15 M2 15...
...,...,...,...,...,...
95,222103531,Оглавление: <@#Модель>MacBook Air 13 M3<@>/8/<...,"{'Модель': 'MacBook Air 13 M3', 'Серия ноутбук...",8,Оглавление: MacBook Air 13 M3/8/512(Midnight)\...
96,14411265,Оглавление: Видеорегистраторы автомобильные <@...,"{'Модель': 'R600 QUAD HD', 'Диагональ экрана':...",9,Оглавление: Видеорегистраторы автомобильные R6...
97,213831515,Оглавление: Планшет <@#Модель>HONOR Pad 8<@> <...,"{'Модель': 'HONOR Pad 8', 'Цвет': 'синий', 'Ти...",9,"Оглавление: Планшет HONOR Pad 8 11.97"" Bluetoo..."
98,215647452,Оглавление: Мышь Defender <@#Модель>Venom GM-6...,"{'Модель': 'Venom GM-640L', 'Цвет': 'черный', ...",9,Оглавление: Мышь Defender Venom GM-640L\nОписа...


In [ ]:
clean[0]

'Оглавление: Роутер Huawei B311-221 с дополнительной внешней антенной +сим карта\nОписание:  Уникальный полный комплект: Wi-Fi-роутер + дополнительная антенна 5 м + сим-карта. Роутер имеет слот для установки SIM-карты и может обеспечить интернет-подключение до 30 беспроводных устройств одновременно.\n\nЭтот комплект является готовым решением для выхода в интернет в тех местах, где отсутствует проводной интернет. Роутер поддерживает работу как с проводным интернетом, так и с мобильным интернетом через сим-карту. Это особенно актуально, если проводной интернет работает с перебоями или вовсе отсутствует.\n\nРоутер поддерживает сети 3G, 4G, LTE 4 категории и имеет 2 порта: LAN и WAN. Он также поддерживает режим репитера и родительский контроль. Размер роутера: длина 18 см, ширина 13 см, высота 2 см.\n\nВ комплекте с роутером идет внешняя усиленная антенна с разъемом SMA и длиной кабеля 5 метров. Сим-карта предоставляется в подарок, и тариф работает по всей России на базе оператора Теле2 в 

In [ ]:
for tag in re.finditer('<@#[^>]+>', descr[0]):
    print(tag)

for tag in re.finditer('<@>', descr[0]):
    print(tag)

<re.Match object; span=(19, 29), match='<@#Модель>'>
<re.Match object; span=(138, 154), match='<@#Комплектация>'>
<re.Match object; span=(181, 197), match='<@#Комплектация>'>
<re.Match object; span=(214, 230), match='<@#Комплектация>'>
<re.Match object; span=(837, 855), match='<@#Длина предмета>'>
<re.Match object; span=(872, 891), match='<@#Ширина предмета>'>
<re.Match object; span=(908, 927), match='<@#Высота предмета>'>
<re.Match object; span=(44, 47), match='<@>'>
<re.Match object; span=(160, 163), match='<@>'>
<re.Match object; span=(208, 211), match='<@>'>
<re.Match object; span=(239, 242), match='<@>'>
<re.Match object; span=(860, 863), match='<@>'>
<re.Match object; span=(896, 899), match='<@>'>
<re.Match object; span=(931, 934), match='<@>'>


In [ ]:
d_data['options'].values[i]

{'Модель': 'iPhone 11 pro',
 'Диагональ экрана': '5.8"',
 'Операционная система': 'iOS',
 'Объем встроенной памяти (Гб)': '64 ГБ',
 'Основная камера (млн. пикс.)': '12 Мп',
 'Вторая основная камера (млн. пикс.)': '12 Мп',
 'Третья основная камера': '12 Мп',
 'Доп. опции камеры': 'Face ID; Распознавание лиц',
 'Процессор': 'Apple A13 Bionic'}

In [ ]:
id_test, sys, us, bot = [], [], [], [] ## Инструкция, Запрос, Ответ
#for i in range(len(train_full)):
for i in range(len(d_data)):
    if d_data.values[i][3] > 5:
        id_test.append(d_data['ids'].values[i])
        sys.append(full_pmt)
        us.append(d_data['clean'].values[i])
        bot.append(d_data['options'].values[i])

    else:
        s = ''
        for j in d_data['options'].values[i]:
            s = s + j + ", "
        s = s[:-2] + ".\n"
        id_test.append(d_data['ids'].values[i])
        sys.append(part_pmt_1 + s + part_pmt_2)
        us.append(d_data['clean'].values[i])
        bot.append(d_data['options'].values[i])

In [ ]:
sys[:10]

['Извлеки атрибуты товара в формате json из описания и оглавления продукта на маркетплейсе. В ответ выведи атрибуты товара и их значения.',
 'Извлеки атрибуты товара в формате json из описания и оглавления продукта на маркетплейсе. В ответ выведи атрибуты товара и их значения.',
 'Извлеки атрибуты товара в формате json из описания и оглавления продукта на маркетплейсе. В ответ выведи атрибуты товара и их значения.',
 'Извлеки атрибуты товара в формате json из описания и оглавления продукта на маркетплейсе. В ответ выведи атрибуты товара и их значения.',
 'Извлеки атрибуты товара в формате json из описания и оглавления продукта на маркетплейсе. В ответ выведи атрибуты товара и их значения.',
 'Извлеки атрибуты товара в формате json из описания и оглавления продукта на маркетплейсе. В ответ выведи атрибуты товара и их значения.',
 'Извлеки атрибуты товара в формате json из описания и оглавления продукта на маркетплейсе. В ответ выведи атрибуты товара и их значения. Выдели только следующи

In [ ]:
print(us[0])

Оглавление: Роутер Huawei B311-221 с дополнительной внешней антенной +сим карта
Описание:  Уникальный полный комплект: Wi-Fi-роутер + дополнительная антенна 5 м + сим-карта. Роутер имеет слот для установки SIM-карты и может обеспечить интернет-подключение до 30 беспроводных устройств одновременно.

Этот комплект является готовым решением для выхода в интернет в тех местах, где отсутствует проводной интернет. Роутер поддерживает работу как с проводным интернетом, так и с мобильным интернетом через сим-карту. Это особенно актуально, если проводной интернет работает с перебоями или вовсе отсутствует.

Роутер поддерживает сети 3G, 4G, LTE 4 категории и имеет 2 порта: LAN и WAN. Он также поддерживает режим репитера и родительский контроль. Размер роутера: длина 18 см, ширина 13 см, высота 2 см.

В комплекте с роутером идет внешняя усиленная антенна с разъемом SMA и длиной кабеля 5 метров. Сим-карта предоставляется в подарок, и тариф работает по всей России на базе оператора Теле2 в сетях 3G

In [ ]:
id_test, sys, us, bot = [], [], [], [] ## Инструкция, Запрос, Ответ
#for i in range(len(train_full)):
for i in range(700):
    id_test.append(train_full['id'].values[i])
    sys.append(full_pmt)
    us.append(train_full['text'].values[i])
    bot.append(list_to_json((train_full['atrs'].values[i])))

#for i in range(len(train_part)):
for i in range(300):
    s = ''
    for j in train_part['new_atrs'].values[i]:
        s = s + j[0] + ", "
    s = s[:-2] + ".\n"
    id_test.append(train_part['id'].values[i])
    sys.append(part_pmt_1 + s + part_pmt_2)
    us.append(train_part['text'].values[i])
    bot.append(list_to_json((train_part['new_atrs'].values[i])))

In [ ]:
id_test.index(221267951)

ValueError: 221267951 is not in list

In [ ]:
list(train_full['id'].values).index('221267951')

ValueError: '221267951' is not in list

In [ ]:
print('hello')

hello


In [ ]:
arr = os.listdir('D:/wb_scraped/tests/yandexFineTunedGptAnswers1000/async/')
gpt_ids = []
gpt_files = []

for f in arr:
    with open('D:/wb_scraped/tests/yandexFineTunedGptAnswers1000/async/' + f, 'r', encoding='utf-8') as myfile:
            data=myfile.read()

    # parse file
    obj = json.loads(data)
    data = obj
    gpt_ids.append(obj['id'])
    gpt_files.append(f.split('async')[0])
    #print(f, f.split('async')[0])
    #print(obj['id'])